In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 03:07:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/06 03:07:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.version

'3.4.2'

In [4]:
!ls -lh fhvhv_tripdata_2021-02.csv

-rw-rw-r-- 1 alexey alexey 700M Oct 29 18:53 fhvhv_tripdata_2021-02.csv


In [4]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [5]:
year = 2019

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/fhv/{year}/{month:02d}/'
    output_path = f'data/pq/fhv/{year}/{month:02d}/'

    df_green = spark.read \
        .option("header", "true") \
        .schema(schema) \
        .csv(input_path)

    df_green \
        .repartition(6) \
        .write.parquet(output_path)

processing data for 2019/1


AnalysisException: [PATH_ALREADY_EXISTS] Path file:/workspaces/data-engineering-zoomcamp-2024/05-batch/code/data/pq/fhv/2019/01 already exists. Set mode as "overwrite" to overwrite the existing path.

**Q3**: How many taxi trips were there on February 15?

In [6]:
from pyspark.sql import functions as F

In [7]:
df = spark.read.parquet('data/pq/fhv/2019/*')

In [8]:
df.registerTempTable('fhv_2019')

/workspaces/spark/spark-3.4.2-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [9]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

**Q4**: Longest trip for each day

In [10]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [11]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2019-07-19|   5743354480|
| 2019-03-25|   2840143080|
| 2019-07-28|   2840142360|
| 2019-05-14|   2840141250|
| 2019-05-12|   2840140992|
+-----------+-------------+



In [13]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG))) AS duration
FROM
    fhv_2019
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+----------+
|pickup_date|  duration|
+-----------+----------+
| 2019-07-19|5743354480|
| 2019-03-25|2840143080|
| 2019-07-28|2840142360|
| 2019-05-14|2840141250|
| 2019-05-12|2840140992|
| 2019-08-09|2272149501|
| 2019-12-11|2272149060|
| 2019-12-16|2272149000|
| 2019-12-18|2272149000|
| 2019-10-28|2272149000|
+-----------+----------+



**Q5**: Most frequent `dispatching_base_num`

How many stages this spark job has?



In [26]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM
    fhv_2019
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02510| 4607586|
|              B02764| 1662983|
|              B02765| 1059883|
|              B02875| 1009567|
|              B02800| 1004974|
+--------------------+--------+



In [19]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|4607586|
|              B02764|1662983|
|              B02765|1059883|
|              B02875|1009567|
|              B02800|1004974|
+--------------------+-------+



**Q6**: Most common locations pair

In [20]:
df_zones = spark.read.parquet('zones')

In [21]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [22]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [23]:
df_zones.registerTempTable('zones')

/workspaces/spark/spark-3.4.2-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [41]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [42]:
spark.sql("""
SELECT
    pul.Zone,
    COUNT(1)
FROM
    fhv_2019 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
GROUP BY
    1
ORDER BY
    2 ASC
LIMIT 5;
""").show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|       Rikers Island|      16|
|Governor's Island...|      23|
|    Great Kills Park|      25|
|         Jamaica Bay|      33|
|Saint Michaels Ce...|     525|
+--------------------+--------+

